# Decision Trees and Ensembles Lab

In this lab we will compare the performance of a simple Decision Tree classifier with a Bagging classifier. We will do that on few datasets, starting from the ones offered by Scikit Learn.

## 1. Breast Cancer Dataset
We will start our comparison on the breast cancer dataset.
You can load it directly from scikit-learn using the `load_breast_cancer` function.

### 1.a Simple comparison
1. Load the data and create X and y
- Initialize a Decision Tree Classifier and use cross_val_score to evaluate it's performance. Set crossvalidation to 5-folds
- Wrap a Bagging Classifier around the Decision Tree Classifier and use cross_val_score to evaluate it's performance. Set crossvalidation to 5-folds. 
- Which score is better? Are the score significantly different? How can you judge that?

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer().data
features = load_breast_cancer().feature_names

In [55]:
data = pd.DataFrame(data, columns = features)
target = pd.Series(load_breast_cancer().target)
load_breast_cancer().target_names

array(['malignant', 'benign'], 
      dtype='<U9')

In [58]:
X = data
y = target

In [63]:
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

dtree_classifier = DecisionTreeClassifier()
dtree_classifier_scores = cross_val_score(dtree_classifier, X, y, cv = 5)
print("CV score for decision tree WITHOUT bagging: {} +/- {}".format(dtree_classifier_scores.mean(), dtree_classifier_scores.std()))

bagging_classifier = BaggingClassifier(base_estimator = dtree_classifier)
bagging_classifier.fit(X, y)
bagged_dtree_classifier_scores = cross_val_score(bagging_classifier, X, y, cv = 5)
print("CV score for decision tree WITH bagging: {} +/- {}".format(bagged_dtree_classifier_scores.mean(), bagged_dtree_classifier_scores.std()))

CV score for decision tree WITHOUT bagging: 0.9103809157368218 +/- 0.015124017881125705
CV score for decision tree WITH bagging: 0.9422085417468258 +/- 0.02968243299944734


### 1.b Scaled pipelines
As you may have noticed the features are not normalized. Do the score improve with normalization?
By now you should be very familiar with pipelines and scaling, so:

1. Create 2 pipelines, with a scaling preprocessing step and then either a decision tree or a bagging decision tree.
- Which score is better? Are the score significantly different? How can you judge that?
- Are the scores different from the non-scaled data?

In [64]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline

In [71]:
dtree_pipe = make_pipeline(RobustScaler(),
                       DecisionTreeClassifier(random_state = 1))

bagged_dtree_pipe = make_pipeline(RobustScaler(),
                        BaggingClassifier(DecisionTreeClassifier(random_state = 1)))

In [72]:
piped_dtree_classifier_scores = cross_val_score(dtree_pipe, X, y, cv = 5)
print("CV score for decision tree WITHOUT bagging: {} +/- {}".format(piped_dtree_classifier_scores.mean(), piped_dtree_classifier_scores.std()))

piped_bagged_dtree_classifier_scores = cross_val_score(bagged_dtree_pipe, X, y, cv = 5)
print("CV score for decision tree WITH bagging: {} +/- {}".format(piped_bagged_dtree_classifier_scores.mean(), piped_bagged_dtree_classifier_scores.std()))

CV score for decision tree WITHOUT bagging: 0.9174913428241631 +/- 0.015840247681111093
CV score for decision tree WITH bagging: 0.9438245479030396 +/- 0.021175888258994805


### 1.c Grid Search

Grid search is a great way to improve the performance of a classifier. Let's explore the parameter space of both models and see if we can improve their performance.

1. Initialize a GridSearchCV with 5-fold cross validation for the Decision Tree Classifier
- search for few values of the parameters in order to improve the score of the classifier
- Use the whole X, y dataset for your test
- Check the best\_score\_ once you've trained it. Is it better than before?
- How does the score of the Grid-searched DT compare with the score of the Bagging DT?
- Initialize a GridSearchCV with 5-fold cross validation for the Bagging Decision Tree Classifier
- Repeat the search
    - Note that you'll have to change parameter names for the base_estimator
    - Note that there are also additional parameters to change
    - Note that you may end up with a grid space to large to search in a short time
    - Make use of the n_jobs parameter to speed up your grid search
- Does the score improve for the Grid-searched Bagging Classifier?
- Which score is better? Are the score significantly different? How can you judge that?

In [73]:
from sklearn.grid_search import GridSearchCV

In [102]:
gs_parameters = {'criterion': ['gini', 'entropy'],
                 'max_depth': [None, 1, 2, 3, 4, 5], 
                 'max_features': [None, 'sqrt', 'auto'], 
                 'class_weight': ['balanced', None]
                }

gs = GridSearchCV(DecisionTreeClassifier(random_state = 1), param_grid = gs_parameters, cv = 5, n_jobs = -1)
gs.fit(X, y)

print(gs.best_params_, "\nBest score: {} +/- {}".format(gs.best_score_.mean(), gs.best_score_.std()))

bagged_gs = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(random_state = 1)), param_grid = {}, cv = 5, n_jobs = -1)
bagged_gs.fit(X, y)

print(gs.best_params_, "\nBest score: {} +/- {}".format(bagged_gs.best_score_.mean(), bagged_gs.best_score_.std()))

{'max_depth': None, 'class_weight': 'balanced', 'criterion': 'gini', 'max_features': None} 
Best score: 0.9420035149384886 +/- 0.0
{'max_depth': None, 'class_weight': 'balanced', 'criterion': 'gini', 'max_features': None} 
Best score: 0.9437609841827768 +/- 0.0


## 2 Diabetes and Regression

Scikit Learn has a dataset of diabetic patients obtained from this study:

http://www4.stat.ncsu.edu/~boos/var.select/diabetes.html
http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf

442 diabetes patients were measured on 10 baseline variables: age, sex, body mass index, average blood pressure, and six blood serum measurements.

The target is a quantitative measure of disease progression one year after baseline.

Repeat the above comparison between a DecisionTreeRegressor and a Bagging version of the same.

### 2.a Simple comparison
1. Load the data and create X and y
- Initialize a Decision Tree Regressor and use cross_val_score to evaluate it's performance. Set crossvalidation to 5-folds. Which score will you use?
- Wrap a Bagging Regressor around the Decision Tree Regressor and use cross_val_score to evaluate it's performance. Set crossvalidation to 5-folds. 
- Which score is better? Are the score significantly different? How can you judge that?

### 2.b Grid Search

Repeat Grid search as above:

1. Initialize a GridSearchCV with 5-fold cross validation for the Decision Tree Regressor
- Search for few values of the parameters in order to improve the score of the regressor
- Use the whole X, y dataset for your test
- Check the best\_score\_ once you've trained it. Is it better than before?
- How does the score of the Grid-searched DT compare with the score of the Bagging DT?
- Initialize a GridSearchCV with 5-fold cross validation for the Bagging Decision Tree Regressor
- Repeat the search
    - Note that you'll have to change parameter names for the base_estimator
    - Note that there are also additional parameters to change
    - Note that you may end up with a grid space to large to search in a short time
    - Make use of the n_jobs parameter to speed up your grid search
- Does the score improve for the Grid-searched Bagging Regressor?
- Which score is better? Are the score significantly different? How can you judge that?


## Bonus: Project 6 data

Repeat the analysis for the Project 6 Dataset